[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tduckenfield/walsa_motionmag_tutorial/blob/main/notebooks/mm_solardata.ipynb)

# Applying motion magnification to solar data

This notebook will demonstrate how to apply motion magnification to EUV imaging data of the solar corona, in the search for the ubiquitous decayless kink oscillations.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
%pip --quiet install dtcwt
import dtcwt

# For those wishing to save their efforts (and avoid memory issues if getting your own FITS), mount your Google Drive.
#from google.colab import drive  # Access google drive files
#drive.mount('/content/gdrive')

In [ ]:
%pip --quiet install sunpy
%pip --quiet install sunpy-soar # This is necessary, do not forget!
import sunpy.map
import sunpy_soar
from sunpy.net import Fido, attrs as a

from astropy.coordinates import SkyCoord
from astropy import units as u

First we must choose some SDO/AIA 171$Å$ data to download. Here I first get a whole sun map and search for an active region, since we are looking for clearly contrasted coronal loops.
As a suggestion, you could try an example of a blowout jet as analysed in [Sukarmadji, Antolin & McLaughlin 2022](https://iopscience.iop.org/article/10.3847/1538-4357/ac7870#apjac7870f4) on  2014 May 3, around 03:10, and look for yourself to see if any small amplitude transverse waves are present!

For this example I have used a random time from last year, and use the [JSOC client](https://docs.sunpy.org/en/v5.0.0/generated/api/sunpy.net.jsoc.JSOCClient.html) to fetch data - feel free to change.

In [ ]:
# I personally dislike Fido...
#time_range = a.Time('2023/06/11 00:00:38', '2023/06/11 00:00:51')
#params_171 = a.Instrument.aia & a.Wavelength(171*u.angstrom)
#params_hmi = a.Instrument.hmi & a.Physobs.los_magnetic_field
#res = Fido.search(time_range, params_171 | params_hmi)  # in this case res[0,0] is 171 & res[1,0] is hmi

from sunpy.net import jsoc
client = jsoc.JSOCClient()

time_range = a.Time('2023/06/11 00:00:38', '2023/06/11 00:00:39')
email = a.jsoc.Notify('tim.duckenfield@northumbria.ac.uk')
params_aia = a.jsoc.Series('aia.lev1_euv_12s') & a.Wavelength(171*u.AA)
params_hmi = a.jsoc.Series('hmi.m_720s') # magnetogram. Maybe change?

response = client.search(time_range, params_aia | params_hmi, email)

Download data.

In [ ]:
# Get data. Use the following command if you are downloading your own data.
# res = client.fetch(response,path='/content/') # Save the full frames 
# Else, skip downloading (and spamming my email notifications!)
!gdown 1fPvZoYQZLWfuK3ZXpECq22YFgI1ygXCD # Full 171 image
!gdown 11hN4HPrP2h7pWRktGXZ9f5BY6ThWu87T # Magnetogram

Now let us take a look at the full disk images. With the `171` dataset we are looking at 17.1 nm extreme ultraviolet light, emitted from Iron nine times ionised (Fe IX) which is roughly 600,000 K. This shows lots of coronal structures such as loops, which is where we will most easily see decayless observations.
With the `hmi` dataset we are looking at a magnetogram, the magnetic field strength along the line of sight at the Sun's surface (photosphere). White indicates the magnetic field is coming towards the observer, black means away. Where there is more complexity in the magnetic field, we will see more coronal structure.

In [ ]:
my_map_171 = sunpy.map.Map('/content/aia.lev1_euv_12s.2023-06-11T000046Z.171.image_lev1.fits')
my_map_hmi = sunpy.map.Map('/content/hmi.m_720s.20230611_000000_TAI.3.magnetogram.fits')

#my_map.quicklook() # SunPy does have a nice quicklook feature!
my_map_171

In [ ]:
# Plot HMI
fig = plt.figure()
ax = fig.add_subplot(projection=my_map_hmi)
my_map_hmi.plot(axes=ax)
plt.colorbar()
plt.show()

In [ ]:
my_map_combined = sunpy.map.Map(my_map_171, my_map_hmi, composite=True)
levels = [-1000, -500, -250, 250, 500, 1000] * u.G
my_map_combined.set_levels(index=1, levels=levels)

# Plot combined image. Note this can take a little while!
fig = plt.figure()
ax = fig.add_subplot(projection=my_map_combined.get_map(0))
my_map_combined.plot(axes=ax)
plt.show()

Now we should extract a suitable [cut out](https://docs.sunpy.org/en/stable/generated/gallery/acquiring_data/downloading_cutouts.html#sphx-glr-generated-gallery-acquiring-data-downloading-cutouts-py). This is because motion magnifying a huge dataset such as full sun images (4096 x 4096) would take a very long time, and likely lead to memory crashes.

We should select a region with clear, visible inhomogeneities which do not move around fast.

In [ ]:
# Specify region of interest.
roi_bottom_left = SkyCoord(Tx=-100*u.arcsec, Ty=0*u.arcsec, frame=my_map_171.coordinate_frame)
roi_top_right = SkyCoord(Tx=100*u.arcsec, Ty=200*u.arcsec, frame=my_map_171.coordinate_frame)
fig = plt.figure()
ax = fig.add_subplot(projection=my_map_171)
my_map_171.plot(axes=ax, clip_interval=(1,99.5)*u.percent)
my_map_171.draw_quadrangle(roi_bottom_left, top_right=roi_top_right, axes=ax, color='C0')
plt.show()

In [ ]:
my_submap = my_map_171.submap(roi_bottom_left, top_right=roi_top_right)

fig = plt.figure()
ax = fig.add_subplot(projection=my_submap)
my_submap.plot(axes=ax)
plt.show()

Now we will fetch a [cutout](https://docs.sunpy.org/en/stable/generated/api/sunpy.net.jsoc.attrs.Cutout.html#sunpy.net.jsoc.attrs.Cutout) of this data.

This requires a registered email address with jsoc (feel free to use mine for this tutorial).

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.visualization import ImageNormalize, SqrtStretch

import sunpy.coordinates  # NOQA
import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a

start_time = Time('2023-06-11T00:00:00', scale='utc', format='isot')
jsoc_email = 'tim.duckenfield@northumbria.ac.uk'

# Set tracking to True and register to True (for smoother tracking)
cutout = a.jsoc.Cutout(roi_bottom_left, top_right=roi_top_right, tracking=True, register=True)
query = Fido.search(
    a.Time(start_time -0.4*u.h, start_time + 0.4*u.h),
    a.Wavelength(171*u.angstrom),
    a.Sample(12*u.s),
    a.jsoc.Series.aia_lev1_euv_12s,
    a.jsoc.Notify(jsoc_email),
    a.jsoc.Segment.image,
    cutout,
)
print(query) # See which files will be downloaded

Now download the data. This can take a little while,depending on the cutout size. I have included my (very crude) code for this example dataset, but I have also saved the fits files in folder `fits/orig_fits`. I will just fetch the output sequence directly later.

In [ ]:
files = Fido.fetch(query)
files.sort()
sq171_20230611 = sunpy.map.Map(files, sequence=True)

# Make a folder to save. 
!mkdir orig_fits  # Should be in content/orig_fits

# Save map sequence as FITS
sq171_20230611.save('/content/orig_fits/20230611_{index:03}.fits')
#sq171_20230611.save('/content/gdrive/MyDrive/orig_fits/20230611_{index:03}.fits')

Now let us visualise this to check it looks okay.

In [ ]:
print("All maps are the same shape: ",sq171_20230611.all_maps_same_shape())
#print("Each of the ",np.size(files_2)," files have a size of ",sq171_20230611[0].shape)
fig = plt.figure()
ax = fig.add_subplot(projection=sq171_20230611[0])
sq171_20230611[0].plot(axes=ax)
plt.colorbar()
plt.show()

Let us save out this map sequence as an mp4.  Remember you can adjust the fps, recall we downloaded 12s cadence data.

In [ ]:
import matplotlib.animation as animation

ani = sq171_20230611.plot()
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='SunPy'), bitrate=1800)
ani.save('/content/20230611_sunmaps.mp4', writer=writer) 

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# If you do not have it, pull the video
!gdown 1K5XrbapQ1MUYt7nsUkY_sNboTne-zz-t

# Show video
mp4 = open('/content/20230611_sunmaps.mp4','rb').read()  #'/content/20230611_sunmaps.mp4'
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Now we have some data, let us convert it to a numpy array, in order to perform the motion magnification.

In [ ]:
# Convert into a numpy array. Note we change order so frames are first.
sq_20230611 = np.transpose(sq171_20230611.as_array(),(2,0,1))
sq_20230611.shape

In [ ]:
# Motion magnify
!git clone https://github.com/Sergey-Anfinogentov/motion_magnification.git
%cd /content/motion_magnification
from magnify import *

k = 5      # Pick a magnification factor
width= 120 # width
sq_20230611_k5w120 = magnify_motions_2d(sq_20230611, k, width) # Be sure to choose appropriate name.

Now we save out the magnified data as FITS files, using the original files' header information. Then we can read those FITS files in to create another map sequence.

I have done this **very** inelegantly! If you know a way of either: avoiding a `for` loop here; extracting the header directly from a map (or map sequence); or adding a header to an array to make a map sequence directly, please let me know!

In [ ]:
# If you are saving out FITS files, highly recommend you use your Google Drive!
#from google.colab import drive  # Access google drive files
#drive.mount('/content/gdrive')

%cd /content/ # Since moved into MM repository. Can use %pwd to check.
!mkdir 

import astropy.io.fits
nframes = sq_20230611_k5w120.shape[0]
for i in range(nframes):
  head = astropy.io.fits.getheader('/content/orig_fits/20230611_'+str(i).zfill(3)+'.fits')
  my_map = sunpy.map.Map(sq_20230611_k5w120[i,:,:] ,head)
  my_map.save('/content/mm_fits/20230611_k5w120_'+str(i).zfill(3)+'.fits',overwrite=True)

In [ ]:
# Make map sequence of the magnified data
files_mm = '/content/mm_fits/20230611_k5w120_*.fits'
sq171_k5w120_20230611 = sunpy.map.Map(files_mm, sequence=True)

In [ ]:
print("All maps are the same shape: ",sq171_k5w120_20230611.all_maps_same_shape())
fig = plt.figure()
ax = fig.add_subplot(projection=sq171_k5w120_20230611[0])
sq171_k5w120_20230611[0].plot(axes=ax)
plt.colorbar()
plt.show()

In [ ]:
ani_mm = sq171_k5w120_20230611.plot()
#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=15, metadata=dict(artist='SunPy'), bitrate=1800)
ani_mm.save('/content/20230611_k5w120_sunmaps.mp4', writer=writer)

In [ ]:
# Show video
mp4 = open('/content/20230611_k5w120_sunmaps.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
np.save('/content/20230611_171.npy',sq_20230611)
np.save('/content/20230611_171_magk5w120.npy',sq_20230611_k5w120)

In [ ]:
# Show videos side by side.
# I suggest using ffmpeg.
!ffmpeg -i 20230611_sunmaps.mp4 -i 20230611_k5w120_sunmaps.mp4 -filter_complex hstack 20230611_comb.mp4

mp4 = open('/content/20230611_comb.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Hopefully you can see decayless oscillations in the motion magnified data, which were (nearly) imperceptible in the original data!

# Thank you!